## Gold: Comercialização

In [0]:
%run "../1 - Setup/Ingestao_bibliotecas_padrao"


In [0]:
from pyspark.sql.functions import col

In [0]:
tabela_silver = spark.table("vitivinicultura.silver_comercializacao.comercio")

In [0]:
display(tabela_silver)

In [0]:
colunas_anos = [str(ano) for ano in range(1970, 2023)]

# 🧩 2️⃣ Cria a expressão stack() para pivotar somente os anos
expr = "stack({n}, {pairs}) as (ano, valor)".format(
    n=len(colunas_anos),
    pairs=", ".join([f"'{ano}', `{ano}`" for ano in colunas_anos])
)

# 🧩 3️⃣ Executa o unpivot da tabela silver
df_unpivot = tabela_silver.selectExpr("Produto", expr)

# 🧩 4️⃣ Converte tipos e remove nulos (opcional)
from pyspark.sql.functions import col

df_unpivot = (
    df_unpivot
    .withColumn("ano", col("ano").cast("int"))
    .withColumn("valor", col("valor").cast("double"))
    .filter(col("valor").isNotNull())
)

In [0]:
%sql
select * from comercio

In [0]:
df_unpivot.createOrReplaceTempView("comercio")
df = spark.sql("""
    SELECT 
        PRODUTO                                                      AS PRODUTO, 
        ANO, 
        TO_DATE(CONCAT('01-01-', CAST(ano AS STRING)), 'dd-MM-yyyy') AS DATA,
        valor                                                        AS LITROS
    FROM comercio
    WHERE PRODUTO IN('VINHO DE MESA','VINHO  FINO DE MESA','VINHO FRIZANTE','VINHO ORGÂNICO','VINHO ESPECIAL')
""")
df.createOrReplaceTempView("comercio_filtrado")

In [0]:
df = spark.sql("""
    SELECT 
        ROW_NUMBER() OVER (ORDER BY E.ANO ASC) AS ID_COMERCIALIZACAO_VINHO,
        E.PRODUTO,
        E.ANO,
        E.DATA,
        E.LITROS
    FROM comercio_filtrado AS E
""")
df.createOrReplaceTempView("COMERCIO_FINAL")

In [0]:
 #Cria tabela gerenciada no Unity Catalog
spark.sql("""
CREATE TABLE IF NOT EXISTS vitivinicultura.gold_comercializacao.comercio_vinhos
""")
# Escrita no formato Delta Lake
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vitivinicultura.gold_comercializacao.comercio_vinhos")

